# Pacchetti

Selezionare il conda environment **camera_cafe**

In [1]:
import pandas as pd # Gestire i dataframe
import numpy as np  # Calcolo di matrici ed altri operazioni matematiche o comodità come np.nan
import re           # Regex per utilizzare le regular expression
import os           # Gestire i file, cambiare percorso file, eliminare file, creare nuova directory ecc...
from tqdm import tqdm # Barra di progresso 
pd.options.mode.chained_assignment=None # Evitare messaggi di errori

import unicodedata # Questo pacchetto serve per normalizzare le stringhe.  
                   # Ad esempio le due stringhe "l'amore è cieco"=="l'amore è cieco"
                   # risultavano diverse, alcuni caratteri nascosti le facevano sembrare diverse in termini di significato
                   # per un umano fossero uguali (si possono capire le differenze tramite il seguente for loop)
                   # for a, b in zip(r"l'amore è cieco", r"l'amore è cieco"):
                   #    print(a, b, a == b)  
                   # Tramite il pacchetto unicodedata si possono normalizzare le stringhe e renderle uguali

#import warnings
#warnings.simplefilter("ignore", category="PerformanceWarning") #Inibire il performance warning

# Importare le funzioni
from  lemmatizzatore import lemmatizzazione

# Importazione 
- Importati i dati;
- Creato un nome univoco per la variabile contenente il numero dell' episodio, **episodio**;
- Creata la variabile **season** per indicare la stagione alla quale un episodio appartiene
- Inserite tutte le stagioni in un solo dataset.

In [ ]:
# Path per la directory nella quale ci sono i file audio delle varie stagioni
path_audio = "/Dati/Trascrizioni"

#Importazione dati prima stagione
df=pd.read_csv("/Dati/primastagione.csv")
df["season"]=1
df=df.rename(columns={'Nº':"episodio"})

#Importazione dati seconda stagione
df2=pd.read_csv("/Dati/secondastagione.csv")
df2["season"]=2
df2=df2.rename(columns={"Nº":"episodio"})
df=pd.concat([df,df2])


#Importazione dati terza stagione
df3=pd.read_csv("/Dati/terzastagione.csv")
df3["season"]=3
df3=df3.rename(columns={'n°':"episodio"})
df=pd.concat([df,df3])


#Importazione dati quarta stagione
df4=pd.read_csv("/Dati/quartastagione.csv")
df4["season"]=4
df4=df4.rename(columns={'n°':"episodio"})
df=pd.concat([df,df4])


#Importazione dati quinta stagione
df5=pd.read_csv("/Dati/quintastagione.csv")
df5["season"]=5
df5=df5.rename(columns={'n°':"episodio"})
df=pd.concat([df,df5])


#Importazione dati sesta stagione
df6=pd.read_csv("/Dati/sestastagione.csv")
df6["season"]=6
df6=df6.rename(columns={'n°':"episodio"})
df=pd.concat([df,df6])
df.reset_index(inplace=True,drop=True)

# Funzioni
Definizione delle funzioni utilizzate 

### text_normalizer

In [3]:
def text_normalizer(testo : str) -> str :
   """ Questa funzione prende una stringa poi restituirla senza spazi aggiuntivi, minuscola e normalizzata. 
       è adatta per uniformare stringhe diverse, facendole usare gli stessi caratteri come apostrofi, accenti...
   """

   testo = testo.replace("'"," ") # Cancellare gli apostrofi
   testo = testo.replace('"'," ") # Cancellare le virgolette
   testo = testo.strip()            # Rimuovere gli spazi all'inizio ed alla fine
 
   testo = re.sub( r'\s{2,}'," ",testo) # Rimuovere gli spazi all'interno della frase più lunghi di un carattere

   testo = testo.lower() # Tutto il testo minuscolo
   testo = re.sub("[, \. ? ! # @ & $ £ % < > | = : ; \+ \* _ / \-  \"  \^ ( ) \{ \} \[ \] \n \\\  ]","",testo)
  
   testo  = unicodedata.normalize('NFC',testo) # Normalizzare  la stringa


   accenti = { # a con accenti
               "à": "a", # A
               "á": "a", # A
               "â": "a", # A
               "ä": "a", # A
               "ã": "a", # A 

               # e con accenti
               "è": "e",
               "é": "e",
               "ê": "e",
               "ë": "e",           

               # i con accenti
               "ì": "i", 
               "í": "i",
               "î": "i",
               "ï": "i",           

               # o con accenti
               "ò": "o",
               "ó": "o",
               "ô": "o",
               "ö": "o",
               "õ": "o", 

               # u con accenti
               "ù": "u",
               "ú": "u",
               "û": "u",
               "ü": "u"    
             }


   # Sostituire gli accenti

   for accento, lettera_senza_accento  in accenti.items():

      testo = testo.replace(accento,lettera_senza_accento)

   return unicodedata.normalize('NFC',testo) # Normalizzare  la stringa di nuovo per evitare errori siccome è stata effettuata una sostituzione di caratteri



# Data Cleaning
- Eliminata la variabile, **'Unnamed: 0'**;
- Riordinate le colonne;
- Resi i nomi delle variabili più user friendly;
- Resi uniformi i codificatori per gli **NaN** nella variabile **guest_star**;
- La variabile **guest_star** presenterà la lista dei nomi.

In [4]:
# Eliminazione della variabile 'Unnamed: 0', risultato dell'importazione dei dati.
df.drop(columns={'Unnamed: 0'},inplace=True)


# Modifica dei nomi del dataframe per renderli più utilizzabili.
df=df.rename(columns={'Guest star':'guest_star','Titolo':'titolo','Trama':'trama','Prima visione':'prima_visione'})

df["trascrizione_puntata"] = None

# Riordinare le colonne.
df=df[['season', 'episodio', 'titolo', 'trama','trascrizione_puntata', 'guest_star', 'prima_visione']]

# Sistemare gli NaN
def na_guest_star(stringa):
    """Funzione per sistemare gli NaN della variabile guest_star.
        Sostituisce i - (trattini) con NaN, così facendo 
        i valori mancanti saranno trattati uniformemente"""
    if stringa=='-':
        return np.NaN
    return stringa
df.guest_star=df.guest_star.apply(lambda x: na_guest_star(x))

# Feature Engineering

Creare nuove variabili da quelle a disposizione ed altro.

## Creare le liste dei nomi per le guest star
In base alla stagione il procedimento sarà diverso.

In [5]:
#Prima stagione,   gueststar doppie non indicate da virgole e trattini
#Seconda stagione, gueststar doppie indicate da virgole, i.split(',')
#Terza stagione,   gueststar doppie indicate da  trattino -  i.split('-')
#Quarta stagione,  gueststar doppie indicate da  trattino -  i.split('-')
#Quinta stagione,  gueststar doppie non indicate, un nome attaccato all'altro.
#Sesta stagione,   gueststar doppie non indicate, un nome attaccato all'altro.
for i,row in df.iterrows():    
    if row.season==2:
        if isinstance(row.guest_star,str):
            df["guest_star"][i]=row.guest_star.split(',')

    elif row.season in [1,3,4]:
        if isinstance(row.guest_star,str):
            df["guest_star"][i]=row.guest_star.split('-')
            
    elif row.season in [5,6]:
        if isinstance(row.guest_star,str):
            da_sostituire=re.findall(r"([a-z][A-Z])",row.guest_star)
            
            for j in da_sostituire:
                row.guest_star=row.guest_star.replace(j,f"{j[0]} - {j[1]}")
            df["guest_star"][i]=row.guest_star.split('-')

## Creare una variabile merge tra titolo e trama
Utilizzare questa variabile per ricerche più approfondite

In [6]:
df["title_trama"]=df.titolo +" " + df.trama

## Creare le variabili binarie per la partecipazione o meno di una guest star

In [7]:
star=set()
for i in df.guest_star:
    if isinstance(i,list):
        for j in i:
            people=j.strip()
            star.add(people)
star=list(star)
star.sort()
for i in star:
    df[f"{i}"]=0
for i,row in df.iterrows():
    if isinstance(row.guest_star,list):
        for j in row.guest_star:
            people=j.strip()
            df.at[i,f"{people}"]=1

/var/folders/j4/9fk3st2d30g22vff83pj6cnw0000gn/T/ipykernel_15156/3686068571.py:10: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[f"{i}"]=0
/var/folders/j4/9fk3st2d30g22vff83pj6cnw0000gn/T/ipykernel_15156/3686068571.py:10: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[f"{i}"]=0
/var/folders/j4/9fk3st2d30g22vff83pj6cnw0000gn/T/ipykernel_15156/3686068571.py:10: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all

## Aggiungere le trascrizioni delle puntate

### Importare le trascrizioni

#### Percorsi file delle trascrizioni
Va automatizzato perchè sono troppe

In [ ]:
# Nome delle cartelle contenente i file audio per ogni stagione
stagioni = {1:'Camera Cafe Prima Stagione',
            2:"Camera Cafe Seconda Stagione",
            3:"Camera Cafe Terza Stagione",
            4:"Camera Cafe Quarta Stagione",
            5:"Camera Cafe Quinta Stagione" # per ora non serve
            }



# Inizializzare un dizionario. In questo dizionario ogni stagione sarà una chiave ed ad essa saranno associati
# tutti i nomi dei file audio appartenenti a quella stagione
puntate = {}

# Parametri di controllo per capire il numero di file audio in totale e per ogni stagione
numero_puntate = 0   # per contare file audio totali
numero_stagione = {} # per contare file audio per stagione

# Iterare sulle stagioni
for key in stagioni.keys():

   # Muoversi alla directory della k-esima stagione
   os.chdir(f"{path_audio}/{stagioni[key]}")

   # Prendere tutti i file nella directory nella k-esima stagione, solo quelli audio (wav)
   
   files = [f"{stagioni[key]}/{file}" for file in os.listdir() if "txt" in file] #QUI ANDRà CAMBIATO SE L'ESTENSIONE DEI FILE AUDIO
   # è DIVERSA DA wav, magari mp4


   numero_puntate += len(files)                    # aggiungere il numero di episodi per la k-esima stagione
   numero_stagione[f"Stagione {key}"] = len(files) # salvare il numero di episodi per la k-esima stagione
   
   # Salvare il nome dei file audio per ogni stagione
   puntate[key] = files

print(f"Ci sono trascrizioni riguardanti {numero_puntate} puntate\n")
print(f"Numero di trascrizioni per ogni stagione:\n  {numero_stagione}")

Ci sono trascrizioni riguardanti 1583 puntate

Numero di trascrizioni per ogni stagione:
  {'Stagione 1': 475, 'Stagione 2': 348, 'Stagione 3': 309, 'Stagione 4': 273, 'Stagione 5': 178}


### Dataframe delle trascrizioni

In [ ]:
# Creare un colonna dei titoli provvisoria, mette tutti i  titoli in minuscolo e toglie gli spazi prima e dopo.   
# In questo modo si facilita il matching tra i nomi degli episodi  indicati nei file txt e quelli del dataframe di wikipedia. Inoltre normalizza la stringa per evitare errori dovuti a caratteri nascosti o diverse codifiche di accenti,
# apostrofi, spazi ecc....
# Questa colonna alla fine di questo processo verrà eliminata
df["titolo_provvisorio"] = df["titolo"].apply(lambda row : text_normalizer(row))

# Iterare sulla stagione
errori = []
e = []
e1 = []
for key in puntate.keys():
   print(f"Estrazione stagione {key}")
   # Estrarre tutte le puntate per la k-esima stagione
   puntate_stagione_k_esima = puntate[key]
   for i in tqdm(range(len(puntate_stagione_k_esima))):
      puntata = puntate_stagione_k_esima[i]

      # Creare il path per il file audio
      file_audio = f"{path_audio}/{puntata}"

      if key == 4: # I file della 4 stagione sono diversi, correggerli
         filename = re.search(r'[^/]+\.txt$', file_audio) 
         filename = filename.group()
         filename = filename.replace(".txt","")
         nome_episodio = text_normalizer(filename)


      else:
         pattern =  r"-\s*(.+)\.txt" # Espressione regolare per estrarre il testo dopo l'ultimo trattino e ignorare l'estensione

         match = re.search(pattern, file_audio) # Trova la corrispondenza file audio - lista file da estrarre
         nome_episodio = text_normalizer(match.group(1)) # Normalizzare la stringa per evitare errori nel match

      # importare la trascrizione per l'iesimo episodio della k-esima puntata
      with open(f"{path_audio}/{puntata}","r") as file_trascrizione :
         trascrizione = file_trascrizione.read()

      # Siccome sicuramente alcuni titoli non combaceranno tra i due dataset
      # inserire un try così da non bloccare il processo.
      # Salvarsi i nomi degli episodi che  danno errore così da risolvere manualmente
      try:
 
         # Trovare la riga corrispondente alla puntata della stagione k-esima 
         # Si ricerca per nome e anche per numero stagione. Numero stagione perchè 
         # alcune stagioni potrebbero avere lo stesso titolo per alcuni episodi (vedi sketch promo)
         indice_riga = df[(df.titolo_provvisorio==nome_episodio) & (df.season==key)].index[0]

         ## Inserire nel dataset principale la trascrizione per quell'episodio 
         df.at[indice_riga,"trascrizione_puntata"] = trascrizione
      except: 
         errori.append(puntata) 

# Eliminare il titolo provvisorio
df.drop(columns=["titolo_provvisorio"],inplace=True)


# Aggiungere alla colonna title_trama anche le trascrizioni, su questa colonna saranno calcolati i vari tf-idf
# Infatti questa colonna sarà composta dal: Titolo episodio, trama episodio, trascrizione episodio
df["title_trama_trascrizione"] = df["title_trama"]+ " " + df["trascrizione_puntata"]

errori

/var/folders/j4/9fk3st2d30g22vff83pj6cnw0000gn/T/ipykernel_15156/2508388458.py:5: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df["titolo_provvisorio"] = df["titolo"].apply(lambda row : text_normalizer(row))


Estrazione stagione 1


100%|██████████| 475/475 [00:02<00:00, 232.94it/s]


Estrazione stagione 2


100%|██████████| 348/348 [00:01<00:00, 242.49it/s]


...

Estrazione stagione 4


100%|██████████| 273/273 [00:01<00:00, 238.21it/s]


Estrazione stagione 5


100%|██████████| 178/178 [00:00<00:00, 232.05it/s]
/var/folders/j4/9fk3st2d30g22vff83pj6cnw0000gn/T/ipykernel_15156/2508388458.py:59: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df["title_trama_trascrizione"] = df["title_trama"]+ " " + df["trascrizione_puntata"]


['Camera Cafe Seconda Stagione/Ep. 346 -  La ventiquattrore.txt']

## Variabili binarie per i personaggi

In [10]:
#In questa lista si possono aggiungere i personaggi per i quali si vuole una binaria assegnata
personaggi = ["Luca","Paolo","Andrea","Ilaria","Alex","Gaia","Silvano","De Marinis","Olmo","Geller","Patty" ,"Lucrezia","Maria Eleonora","Anna","Emma","Giovanna","Presidente","Pippo","Anselmo", "Gloria","Jessica","Lello","Valeria","Brad", "Jonathan", "Signora Bollini", "Mamma di Silvano", "Wanda", "Vittorio","Carminati", "Caterina", "Pooh", "Riccardo Fogli","Roberto" ]

#Vi sono elencati solo i nomi standard dei personaggi.    
#Nella funzione di lemmatizzazione ogni forma ambigua viene ricodificata nella forma standard.   
#Ad esempio 'direttore' in De Marinis, oppure Bitta in Paolo, Nervi in Luca ecc...
#Ovviamente il caso De Marinis può causare ambiguità perchè la parola direttore può essere utilizzata anche per indicare altri personaggi, tuttavia questo è un compromesso 
#in modo da cercare di identificare meglio il personaggio di De Marinis. 

# Ottenere la lista dei personaggi per ogni episodio ed anche tutte le parole lemmatizzate   per ogni episodio, tornerà utile nel file 02
df['list_words_all'],df['Personaggi']= zip(*df['title_trama_trascrizione'].apply(lambda row: lemmatizzazione(row) if type(row) != float else ([], [])))

# Inizializzare tutte le variabili binarie per ogni personaggio
for i in personaggi:
    df[f"{i}"]=0

# Iterare sul dataset
for i,row in df.iterrows():

   # Estrarre la lista dei personaggi per il determinato episodio (riga)
   lista_personaggi = row.Personaggi

   # Pulire la lista dei personaggi in modo da estrarre solo i nomi (non serve, utilizzare se la lista non viene riconosciuta e si lavora con una stringa.split(","))
   #lista_personaggi= set([ re.sub(r"[^a-zA-Z ]", "", personaggio) for personaggio in lista_personaggi])
  
   # Iterare per ogni personaggio nella lista dei personaggi definita in precedenza
   for personaggio in personaggi:

      # Se un personaggio è nella lista dei personaggi della riga(episodio) assegnare 1 alla sua variabile binaria (solo a quella riga/episodio)
      if personaggio in lista_personaggi:
         df.at[i,personaggio]=1

/var/folders/j4/9fk3st2d30g22vff83pj6cnw0000gn/T/ipykernel_15156/3967406166.py:11: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df['list_words_all'],df['Personaggi']= zip(*df['title_trama_trascrizione'].apply(lambda row: lemmatizzazione(row) if type(row) != float else ([], [])))
/var/folders/j4/9fk3st2d30g22vff83pj6cnw0000gn/T/ipykernel_15156/3967406166.py:11: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df['list_words_all'],df['Personaggi']= zip(*df['title_trama_trascrizione'].apply(lambda row: lemmatizzazione(row) if type(ro

# Ricodifica nomi guest star che davano errori

In [11]:
df = df.rename(columns={'Andrea "Ahmed" Orancone':'Andrea Orancone'})

# Esportazione dati

In [ ]:
df.to_csv("/Dati/dati_puliti.csv") #esportazione dati

#esportazione nomi gueststar
with open("/Dati/lista_gueststar.txt", 'w+') as v:
    for i in star:
        v.write(f'{i},')